<a href="https://colab.research.google.com/github/Mia1011/DS/blob/main/LangChain%20%26%20HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用 LangChain 與 Hugging Face 創建 LLM 流程 Part1

1. 安裝 Hugging Face Hub 庫
2. 導入 Hugging Face Hub 庫
3. 執行 notebook_login() 函數

```
# This is formatted as code
```



In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ruiiw/k-12_education_data2")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 717
    })
})

In [ ]:
dataset['train']['text'][0]

'<s>[INST] You will grade a student\'s writing that completes a provided partial story. The student is expected to complete the story and follow the leading sentences in the paragraphs. Grade the writing in the 3 categories (Content, Grammar and vocabulary, Structure and organization) on a 0-8.33 scale and provide the total score: Over the following rehearsals(排练). Avery kept ignoring me on purpose.. our tension\\nwas like a sword hanging above me until that\\nWednesday, when the rehearsal went to Dormouse\'s part.\\nIt seemed that Avery was so nervous that she made several\\nmistakes. Concern seizing my heart, I ran to comfort her.\\nregardless of the tension between us. "Calm down, Avery\\nTake a deep breath, you can make it:" A flash of astonishm\\nent\\nand gratitude rushed through her eyes. Hesitant for a while\\nalso\\nI told her that I was also feen with enly when she won the karate\\nchampionship and asked if we can practice our lyrics together\\nshe l smiled LL think ! might b

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

messages = [
    {"role": "user", "content": dataset['train']['text'][0] }
]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


In [ ]:
from transformers import BartTokenizer, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartModel.from_pretrained('facebook/bart-base')

inputs = tokenizer(dataset['train']['text'][0], return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
print(last_hidden_states)

tensor([[[ 3.2578,  2.4591,  1.3512,  ...,  2.0756,  0.5259, -0.4182],
         [-3.5348, -0.3611, -3.2727,  ...,  2.0167,  0.1096, -0.6727],
         [-3.3876, -0.4095, -3.0529,  ...,  2.1771,  1.0476, -0.6221],
         ...,
         [-0.4125,  0.9979, -1.4725,  ...,  0.1727,  0.2372,  0.0175],
         [-0.8175, -1.5013, -1.9991,  ...,  2.1212, -0.8350,  0.6183],
         [-0.0298, -0.3790, -0.7800,  ...,  0.2120,  0.9384,  0.1025]]],
       grad_fn=<NativeLayerNormBackward0>)


# Fine-Tune 流程如下：

1. 安裝 Hugging Face Transformers 庫。
2. 下載 BLOOM 模型。
3. 將您的文字數據轉換為 Hugging Face Transformers 兼容的格式。
4. 創建一個 fine-tuning 數據集。
5. 創建一個 fine-tuning 模型。
6. 訓練 fine-tuning 模型。
7. 評估 fine-tuning 模型。

https://www.huaxiaozhuan.com/%E5%B7%A5%E5%85%B7/huggingface_transformer/chapters/4_trainer.html


In [ ]:
!pip install transformers

In [ ]:
your_text_data = dataset['train']['text']

In [ ]:
print(your_text_data)

['<s>[INST] You will grade a student\'s writing that completes a provided partial story. The student is expected to complete the story and follow the leading sentences in the paragraphs. Grade the writing in the 3 categories (Content, Grammar and vocabulary, Structure and organization) on a 0-8.33 scale and provide the total score: Over the following rehearsals(排练). Avery kept ignoring me on purpose.. our tension\\nwas like a sword hanging above me until that\\nWednesday, when the rehearsal went to Dormouse\'s part.\\nIt seemed that Avery was so nervous that she made several\\nmistakes. Concern seizing my heart, I ran to comfort her.\\nregardless of the tension between us. "Calm down, Avery\\nTake a deep breath, you can make it:" A flash of astonishm\\nent\\nand gratitude rushed through her eyes. Hesitant for a while\\nalso\\nI told her that I was also feen with enly when she won the karate\\nchampionship and asked if we can practice our lyrics together\\nshe l smiled LL think ! might 

In [ ]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("facebook/bart-base")

encoded_data = tokenizer.batch_encode_plus(
    your_text_data,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512,
)

In [ ]:
print(encoded_data)

{'input_ids': tensor([[    0,     0, 10975,  ...,     1,     1,     1],
        [    0,     0, 10975,  ...,     1,     1,     1],
        [    0,     0, 10975,  ...,     1,     1,     1],
        ...,
        [    0,     0, 10975,  ...,     1,     1,     1],
        [    0,     0, 10975,  ...,     1,     1,     1],
        [    0,     0, 10975,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("Alireza1044/dwight_bert_lm")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at Alireza1044/dwight_bert_lm and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate -U
!pip show accelerate

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    train_dataset=your_text_data,
    args=transformers.TrainingArguments(
        output_dir="output_dir",
        num_train_epochs=10,
    ),
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
!pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2

In [ ]:
!pip install langchain_community

In [ ]:
!pip install transformers

In [ ]:
!pip show transformers

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 10},
)

In [ ]:
# 生成文本
inputs = {"prompt": "Hello, my dog is cute"}
outputs = hf(**inputs)

print(outputs)

In [ ]:
!pip install langchain[all]

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import os
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=128,
    temperature=0.5,
    token=HUGGINGFACEHUB_API_TOKEN
)

In [ ]:
# 生成文本
prompt_template = "discuss {prompt} step by step"

prompt = PromptTemplate.from_template(prompt_template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
inputs = {"Hello, my cat is cute"}
print(llm_chain.run(inputs))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




1. Greeting: The bot greets you with a friendly response.
2. Understanding: The bot recognizes the statement as an expression of affection towards a cat.
3. Processing: The bot acknowledges the cuteness of cats and the speaker's attachment to their own cat.
4. Response: The bot expresses agreement and shares a common interest in cats.

Example conversation:

Bot: Hello! It's nice to meet you.
Speaker: Hi, my cat is so cute.
Bot: Cats can indeed be very adorable! I love cats too. What's your cat's name?
